In [116]:
from logicqubit.logic import *
from logicqubit.zhegalkin import *
import numpy as np

In [121]:
# a=110 f(x)=f(x+a)
# Uf*H|x1,x2,x3>|000>
# x1 x2 x3 y1 y2 y3
# 0  0  0  1  0  1
# 0  0  1  0  1  0
# 0  1  0  0  0  0
# 0  1  1  1  1  0
# 1  0  0  0  0  0
# 1  0  1  1  1  0
# 1  1  0  1  0  1
# 1  1  1  0  1  0

def simons3q():
    logicQuBit = LogicQuBit(7, first_left=True)

    x1 = Qubit()
    x2 = Qubit()
    x3 = Qubit()

    y1 = Qubit()
    y2 = Qubit()
    y3 = Qubit()

    x1.H()
    x2.H()
    x3.H()

    oracle = Oracle([x1, x2, x3])
    oracle.addTable(y1, ['000','011','101','110'])
    oracle.addTable(y2, ['001','011','101','111'])
    oracle.addTable(y3, ['000','110'])

    logicQuBit.addOracle(oracle)
    logicQuBit.Measure_One(y1)[0]
    logicQuBit.Measure_One(y2)[0]
    logicQuBit.Measure_One(y3)[0]
    
    x1.H()
    x2.H()
    x3.H()
    
    m_x1 = logicQuBit.Measure_One(x1)[0]
    m_x2 = logicQuBit.Measure_One(x2)[0]
    m_x3 = logicQuBit.Measure_One(x3)[0]
    return [m_x1, m_x2, m_x3]

In [122]:
#res = logicQuBit.Measure([y1,y2,y3], True)
#logicQuBit.Plot()

In [124]:
result = [simons3q() for i in range(6)]
print(result)

[[1, 1, 1], [1, 1, 1], [1, 1, 0], [1, 1, 0], [0, 0, 1], [1, 1, 1]]


In [93]:
#0,0,1
#1,1,0
#1,1,1
#a=110
# a*z = 1*x1 + 1*x2 + 0*x3

In [117]:
# https://gist.github.com/popcornell/bc29d1b7ba37d824335ab7b6280f7fec#file-gf2elim-py
def gf2elim(M):

    m,n = M.shape

    i=0
    j=0

    while i < m and j < n:
        # find value and index of largest element in remainder of column j
        k = np.argmax(M[i:, j]) +i

        # swap rows
        #M[[k, i]] = M[[i, k]] this doesn't work with numba
        temp = np.copy(M[k])
        M[k] = M[i]
        M[i] = temp


        aijn = M[i, j:]

        col = np.copy(M[:, j]) #make a copy otherwise M will be directly affected

        col[i] = 0 #avoid xoring pivot row with itself

        flip = np.outer(col, aijn)

        M[:, j:] = M[:, j:] ^ flip

        i += 1
        j +=1

    return M

In [125]:
M=np.array(result)
gf2elim(M)

array([[1, 1, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])